In [4]:
!! pip install --upgrade transformers
!! pip install tf-keras
! pip install optuna
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [7]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import TFBertForSequenceClassification, BertTokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import optuna
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [9]:
tf.config.list_physical_devices('GPU')

2024-11-01 12:41:07.232959: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-01 12:41:20.202506: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-01 12:41:20.206234: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [22]:
df = pd.read_csv("Features_For_Traditional_ML_Techniques.csv", index_col=0)
# subset_data = df.sample(frac=0.1, random_state=42)

In [23]:
len(df)

134198

In [24]:
def prepare_dataset(texts, labels, tokenizer, batch_size=32, max_length=64):
    # Convert texts to list if it's numpy array
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()
        
    # Ensure labels are the right shape
    labels = np.array(labels)
    if len(labels.shape) == 1:
        labels = labels.reshape(-1, 1)
    
    encodings = tokenizer(
        texts,
        truncation=True,
        padding=True,
        return_tensors='tf',
        max_length=max_length
    )
    
    dataset = tf.data.Dataset.from_tensor_slices((
        {
            'input_ids': encodings['input_ids'],
            'attention_mask': encodings['attention_mask']
        },
        labels
    ))
    dataset = dataset.cache()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    df['tweet'].values,
    df['majority_target'].values,
    test_size=0.2,
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size = 0.2,
    random_state=42
)

# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [26]:
def objective(trial, X_train, y_train, X_val, y_val, X_test, y_test, tokenizer):
    """Optuna objective function to minimize"""
    
    # Get hyperparameters for this trial
    params = {
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
        'max_length': trial.suggest_categorical('max_length', [64, 128, 256]),
        'learning_rate': trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True),
        # 'lr_reduction_factor': trial.suggest_float('lr_reduction_factor', 0.1, 0.5),
        'epochs': 8  # Fixed number of epochs
    }
    
    # Prepare datasets
    train_dataset = prepare_dataset(
        X_train, 
        y_train, 
        tokenizer, 
        batch_size=params['batch_size'],
        max_length=params['max_length']
    )
    val_dataset = prepare_dataset(
        X_val, 
        y_val, 
        tokenizer, 
        batch_size=params['batch_size'],
        max_length=params['max_length']
    )
    test_dataset = prepare_dataset(
        X_test, 
        y_test, 
        tokenizer,
        batch_size=params['batch_size'],
        max_length=params['max_length']
    )
    
    # Initialize model
    model = TFBertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=1
    )
    
    # Compile model
    optimizer = Adam(learning_rate=params['learning_rate'])
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=4,
        restore_best_weights=True
    )
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=1e-6
    )
    
    # Train model
    history = model.fit(
        train_dataset,
        epochs=params['epochs'],
        validation_data=val_dataset,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(test_dataset, verbose=0)
    y_pred = model.predict(test_dataset)
    logits = y_pred.logits
    
    # Get predictions
    if len(logits.shape) == 1:
        y_pred_classes = (logits > 0.5).astype(int).flatten()
    else:
        y_pred_classes = np.argmax(logits, axis=1)
    
    # Get metrics
    report = classification_report(y_test, y_pred_classes, output_dict=True)
    
    # Store trial results
    trial.set_user_attr('test_loss', test_loss)
    trial.set_user_attr('test_accuracy', test_accuracy)
    trial.set_user_attr('precision', report["weighted avg"]["precision"])
    trial.set_user_attr('recall', report["weighted avg"]["recall"])
    trial.set_user_attr('f1_score', report["weighted avg"]["f1-score"])
    trial.set_user_attr('best_epoch', len(history.history['loss']))
    trial.set_user_attr('min_val_loss', min(history.history['val_loss']))
    
    return report["weighted avg"]["f1-score"]

In [27]:
def run_optimization(X_train, y_train, X_val, y_val, X_test, y_test, n_trials=20):
    
    print("Starting hyperparameter optimization...")
    print(f"Data shapes:")
    print(f"X_train: {X_train.shape}")
    print(f"y_train: {y_train.shape}")
    print(f"X_val: {X_val.shape}")
    print(f"y_val: {y_val.shape}")
    print(f"X_test: {X_test.shape}")
    print(f"y_test: {y_test.shape}")
    
    # Create study object
    study = optuna.create_study(direction='maximize')
    
    # Run optimization
    study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val, X_test, y_test, tokenizer),
                  n_trials=n_trials)
    
    # Collect results
    results = []
    for trial in study.trials:
        if trial.state == optuna.trial.TrialState.COMPLETE:
            result = {
                'batch_size': trial.params['batch_size'],
                'max_length': trial.params['max_length'],
                'learning_rate': trial.params['learning_rate'],
                # 'lr_reduction_factor': trial.params['lr_reduction_factor'],
                'test_loss': trial.user_attrs['test_loss'],
                'test_accuracy': trial.user_attrs['test_accuracy'],
                'precision': trial.user_attrs['precision'],
                'recall': trial.user_attrs['recall'],
                'f1_score': trial.user_attrs['f1_score'],
                'best_epoch': trial.user_attrs['best_epoch'],
                'min_val_loss': trial.user_attrs['min_val_loss']
            }
            results.append(result)
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('f1_score', ascending=False)
    
    # Save results
    results_df.to_csv('optuna_hyperparameter_results.csv', index=False)
    
    # Print best trial information
    print("\nBest trial:")
    trial = study.best_trial
    print(f"Value (F1 Score): {trial.value:.4f}")
    print("\nBest hyperparameters:")
    for key, value in trial.params.items():
        print(f"{key}: {value}")
    
    # Create visualizations
    try:
        import plotly
        fig = optuna.visualization.plot_optimization_history(study)
        fig.write_html("optimization_history.html")
        
        fig_importance = optuna.visualization.plot_param_importances(study)
        fig_importance.write_html("parameter_importance.html")
    except Exception as e:
        print(f"Could not generate plots: {str(e)}")
    
    return results_df, study

In [28]:
# Run the optimization
results_df, study = run_optimization(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    n_trials=20
)

# Display top 5 results
print("\nTop 5 configurations:")
print(results_df.head().to_string())

# Access best parameters
best_params = study.best_params
print("\nBest parameters:", best_params)

# Access best score
best_score = study.best_value
print("Best F1 score:", best_score)

[I 2024-11-01 13:10:09,208] A new study created in memory with name: no-name-ceabffb0-d9fa-40a4-914e-3d488b8cb203


Starting hyperparameter optimization...
Data shapes:
X_train: (85886,)
y_train: (85886,)
X_test: (26840,)
y_test: (26840,)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/8


2024-11-01 13:12:48.646479: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 192.00MiB (rounded to 201326592)requested by op tf_bert_for_sequence_classification_1/bert/encoder/layer_._6/attention/self/Softmax
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-11-01 13:12:48.646541: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2024-11-01 13:12:48.646551: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 164, Chunks in use: 164. 41.0KiB allocated for chunks. 41.0KiB in use in bin. 1.2KiB client-requested in use in bin.
2024-11-01 13:12:48.646557: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in 

ResourceExhaustedError: Graph execution error:

Detected at node tf_bert_for_sequence_classification_1/bert/encoder/layer_._6/attention/self/Softmax defined at (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/ipykernel/__main__.py", line 5, in <module>

  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/home/ec2-user/anaconda3/envs/tensorflow2_p310/